In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import time
from main import *
from torch.utils.data import DataLoader
from models.deepsets import *

In [2]:
data_1D = DataLoader(AntDataset('data/large/scaled/YZ_Large_testing.npz'))
data_2D = DataLoader(AntDataset2D('data/large/scaled/YZ_Large_testing.npz'))

In [3]:
for i in data_1D:
    print(i[1])
    break

tensor([-0.1296])


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
NN1 = SimpleNN(2048, 1, 8, nn.ReLU)
NN1.load_state_dict(torch.load('weights/FFNN/1NN_Weight.pt'))

NN_2 = NN2(2048, 1, 8, 8, nn.ReLU)
NN_2.load_state_dict(torch.load('weights/FFNN/2NN_Weight.pt'))

DeepSet = DeepSetRegression(2, 1,1, 'robust_mean', 64)
DeepSet.load_state_dict(torch.load('weights/DeepSet/DeepSet_Weight.pt'))

NN1_time = []
NN2_time = []
DeepSet_time = []

NN1.eval()
NN_2.eval()
for i, data in enumerate(data_1D):
    if i == 1000:
        break
    start_time = time.time()
    with torch.no_grad():
        output = NN1(data[0])
    end_time = time.time()
    NN1_time.append((end_time - start_time) * 1000)

    start_time = time.time()
    with torch.no_grad():
        output = NN_2(data[0])
    end_time = time.time()
    NN2_time.append((end_time - start_time) * 1000)

DeepSet.eval()
for i, data in enumerate(data_2D):
    if i == 1000:
        break
    start_time = time.time()
    with torch.no_grad():
        output = DeepSet(data[0])
    end_time = time.time()
    DeepSet_time.append((end_time - start_time) * 1000)

print(f"NN1\nMean: {np.mean(NN1_time)}\nSTD: {np.std(NN1_time)}")
print()
print(f"NN2\nMean: {np.mean(NN2_time)}\nSTD: {np.std(NN2_time)}")
print()
print(f"DeepSet\nMean: {np.mean(DeepSet_time)}\nSTD: {np.std(DeepSet_time)}")

NN1
Mean: 0.050290584564208984
STD: 0.0146992733175466

NN2
Mean: 0.05852699279785156
STD: 0.015308722363402914

DeepSet
Mean: 0.4764091968536377
STD: 0.05850663635241254
